<a href="https://colab.research.google.com/github/JulianBeaupre/jube1/blob/master/Python_Exercise_1_Basic_Ad_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

DATA_URL = 'http://128.138.93.164/aprd6342/data/TravelPonyFacebook.csv'
df = pd.read_csv(DATA_URL)

In [0]:
df.head()


,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857


In [0]:
!pip install arrow

ADD COST PER IMPRESSION COLUMN TO DATABASE


In [0]:
CPI = df['Amount Spent (USD)']/df['Impressions']
CPI

0       0.000466
1       0.000652
2       0.000448
3       0.000452
4       0.002464
          ...   
3700    0.003512
3701    0.002936
3702    0.008919
3703    0.006643
3704    0.000439
Length: 3705, dtype: float64

In [0]:
df['CPI'] = CPI
df

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),CPI
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0.000466
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,0.000652
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,0.000448
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,0.000452
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,0.002464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3700,2013-06-11,2013-06-11,Bob Jackson,Ad 1,298,1.003356,299,2,1,1.05,0,1,1,1,0,0,0,0,0.00,1.050000,0.003512
3701,2013-06-10,2013-06-10,Bob Jackson,Ad 1,514,1.060311,545,3,3,1.60,0,3,3,3,0,0,0,0,0.00,0.533333,0.002936
3702,2013-06-09,2013-06-09,Bob Jackson,Ad 1,111,1.000000,111,1,1,0.99,0,1,1,1,0,0,0,0,0.00,0.990000,0.008919
3703,2013-06-08,2013-06-08,Bob Jackson,Ad 1,143,1.000000,143,3,3,0.95,0,3,3,3,0,0,0,0,0.00,0.316667,0.006643


EXTRACT DAY OF THE WEEK

In [0]:
df['Start Date']=pd.to_datetime(df['Start Date'])
df['weekday']= df['Start Date'].apply(lambda time: time.strftime('%A'))
df

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),CPI,weekday
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0.000466,Monday
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,0.000652,Sunday
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,0.000448,Saturday
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,0.000452,Friday
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,0.002464,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3700,2013-06-11,2013-06-11,Bob Jackson,Ad 1,298,1.003356,299,2,1,1.05,0,1,1,1,0,0,0,0,0.00,1.050000,0.003512,Tuesday
3701,2013-06-10,2013-06-10,Bob Jackson,Ad 1,514,1.060311,545,3,3,1.60,0,3,3,3,0,0,0,0,0.00,0.533333,0.002936,Monday
3702,2013-06-09,2013-06-09,Bob Jackson,Ad 1,111,1.000000,111,1,1,0.99,0,1,1,1,0,0,0,0,0.00,0.990000,0.008919,Sunday
3703,2013-06-08,2013-06-08,Bob Jackson,Ad 1,143,1.000000,143,3,3,0.95,0,3,3,3,0,0,0,0,0.00,0.316667,0.006643,Saturday


In [0]:
day=df.groupby('weekday')['CPI'].mean()
day

weekday
Friday       0.004097
Monday       0.002910
Saturday     0.002629
Sunday       0.003688
Thursday     0.003398
Tuesday      0.003040
Wednesday    0.002981
Name: CPI, dtype: float64

In [0]:
day.nlargest(1)

weekday
Friday    0.004097
Name: CPI, dtype: float64

THE DAY THAT IS THE MOST EXPENSIVE TO GENERATE IMPRESSIONS IS FRIDAY

In [0]:
day.nsmallest(1)

weekday
Saturday    0.002629
Name: CPI, dtype: float64

THE BEST(CHEAPEST) DAY TO GENERATE IMPRESSIONS IS SATURDAY

##CORRELATIONS##

In [0]:
correl = df.corr()
correl

,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),CPI
Reach,1.000000,0.334101,0.966851,0.709466,0.722249,0.703124,0.304388,0.597648,0.605244,0.151727,0.100254,0.153288,0.205678,0.858860,0.646366,0.168697,0.028005
Frequency,0.334101,1.000000,0.386949,0.128473,0.135103,0.130201,0.000182,0.111645,0.119780,-0.015511,-0.008028,-0.012113,-0.009054,0.240826,0.154855,0.043232,-0.018217
Impressions,0.966851,0.386949,1.000000,0.538032,0.553394,0.532913,0.158246,0.459510,0.474330,0.065275,0.043358,0.067018,0.090544,0.769145,0.565503,0.091639,0.006973
Clicks,0.709466,0.128473,0.538032,1.000000,0.998287,0.883411,0.591499,0.797945,0.786167,0.465285,0.242256,0.407737,0.448348,0.817139,0.594057,0.209975,0.088315
Unique Clicks,0.722249,0.135103,0.553394,0.998287,1.000000,0.882993,0.584614,0.787337,0.775598,0.445915,0.224601,0.379449,0.432454,0.822962,0.597671,0.213844,0.086798
Amount Spent (USD),0.703124,0.130201,0.532913,0.883411,0.882993,1.000000,0.757612,0.819253,0.789348,0.386383,0.259781,0.420018,0.479713,0.817490,0.637906,0.323046,0.119201
Page Likes,0.304388,0.000182,0.158246,0.591499,0.584614,0.757612,1.000000,0.650871,0.579933,0.454491,0.290983,0.432091,0.402059,0.477790,0.304196,0.271398,0.143931
Page Engagement,0.597648,0.111645,0.459510,0.797945,0.787337,0.819253,0.650871,1.000000,0.995945,0.700280,0.387753,0.648593,0.814885,0.731632,0.617181,0.147983,0.117141
Post Engagement,0.605244,0.119780,0.474330,0.786167,0.775598,0.789348,0.579933,0.995945,1.000000,0.697600,0.381613,0.644780,0.826786,0.728476,0.626231,0.126639,0.108649
Post Likes,0.151727,-0.015511,0.065275,0.465285,0.445915,0.386383,0.454491,0.700280,0.697600,1.000000,0.424246,0.689834,0.639658,0.240632,0.267482,0.058336,0.106425


In [0]:
specifications=correl.iloc[[0,1,4,6],5]
specifications

Reach            0.703124
Frequency        0.130201
Unique Clicks    0.882993
Page Likes       0.757612
Name: Amount Spent (USD), dtype: float64

In [0]:
maxspec = specifications.nlargest(1)
maxspec

Unique Clicks    0.882993
Name: Amount Spent (USD), dtype: float64

THE STRONGEST CORRELATIONS IS UNIQUE CLICKS. THIS MEANS THAT AS AMOUNT SPENT INCREASES, UNIQUE CLICKS WILL INCREASE MORE THAN THE OTHER THREE VARIABLES.

##LINEAR REGRESSION##

In [0]:
import statsmodels.api as sm

In [0]:
dv=df['Unique Clicks']

In [0]:
iv=df[['Reach','Frequency']]

In [0]:
l=sm.OLS(dv,iv)

In [0]:
r=l.fit()

In [0]:
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Thu, 14 Nov 2019   Prob (F-statistic):                        0.00
Time:                        04:05:37   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

FREQUENCY PREDICTS UNIQUE CLICKS BETTER THAN REACH. THIS MEANS THAT AN INCREASE IN FREQUENCY IS MORE LIKELY TO GENERATE UNIQUE CLICKS